In [ ]:
from qdrant_client import QdrantClient, models
import os

client = QdrantClient(url="http://localhost:6333")

collection_name = "vector_search_fundamentals"

In [ ]:

client.create_collection(
    collection_name= collection_name,
    vectors_config={
        "image": models.VectorParams(size=4, distance=models.Distance.DOT),
        "text": models.VectorParams(size=5, distance=models.Distance.COSINE),
    },
    sparse_vectors_config={
        "text-sparse": models.SparseVectorParams()
    },
)

True

In [4]:
client.upsert(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=1,
            vector={
                "image": [0.9, 0.1, 0.1, 0.2],
                "text": [0.4, 0.7, 0.1, 0.8, 0.1],
                "text-sparse": {
                    "indices": [1, 3, 5, 7],
                    "values": [0.1, 0.2, 0.3, 0.4],
                },
            },
        ),
    ],
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
models.Filter(
    should=[
        models.Filter(
            must=[
                models.FieldCondition(key="category", match=models.MatchValue(value="electronics")),
                models.FieldCondition(key="price", range=models.Range(lt=200))
            ]),
        models.Filter(
            must=[
                models.FieldCondition(key="category", match=models.MatchValue(value="books")),
                models.FieldCondition(key="rating", range=models.Range(gte=4.0))
            ])
    ]
)

### Advanced Filtering: Nested Objects ###

In [ ]:
# Consider a product with multiple reviews:
{
  "id": 1,
  "product": "Laptop",
  "reviews": [
    {"user": "alice", "rating": 5, "verified": true},
    {"user": "bob", "rating": 3, "verified": false}
  ]
}

# To find products with verified 5-star reviews (both conditions must apply to the same review):
models.Filter(
    must=[
        models.NestedCondition(
            nested=models.Nested(
                key="reviews",
                filter=models.Filter(
                    must=[
                        models.FieldCondition(key="rating", match=models.MatchValue(value=5)),
                        models.FieldCondition(key="verified", match=models.MatchValue(value=True))
                    ])
            )
        )
    ]
)

In [5]:
# Index frequently filtered fields
client.create_payload_index(
    collection_name=collection_name,
    field_name="category",
    field_schema=models.PayloadSchemaType.KEYWORD,
)

# For multi-tenant applications, mark tenant fields
client.create_payload_index(
    collection_name=collection_name,
    field_name="tenant_id",
    field_schema=models.KeywordIndexParams(type="keyword", is_tenant=True),
)

UpdateResult(operation_id=5, status=<UpdateStatus.COMPLETED: 'completed'>)

### Search with Metadata ###

In [1]:
def search_with_filters(query, document_type=None, date_range=None):
    """Search with metadata filtering"""

    # Build filter conditions
    filter_conditions = []

    if document_type:
        filter_conditions.append(
            models.FieldCondition(
                key="source_type", match=models.MatchValue(value=document_type)
            )
        )

    if date_range:
        filter_conditions.append(
            models.FieldCondition(
                key="created_at",
                range=models.Range(gte=date_range["start"], lte=date_range["end"]),
            )
        )

    # Execute search
    query_filter = models.Filter(must=filter_conditions) if filter_conditions else None

    results = client.query_points(
        collection_name="documents",
        query=generate_embedding(query),
        query_filter=query_filter,
        limit=5,
    )

    return results